Ejecutar la siguiente instrucion si estas trabajando en google colab

In [ ]:
#!pip install psycopg2-binary

In [2]:
import psycopg2 as pg2
from prettytable import from_db_cursor, PrettyTable
import reprlib as rep
try:
    con = pg2.connect(
    host = "ec2-3-222-74-92.compute-1.amazonaws.com",
    database = "ddkqm651ptq41g",
    user = "knxfxnexhnvtqx",
    password = "fd99404f7a0e7a8c5fe119ab7ab20ef7a0bea0b3c7fe2bb262b0c0e8b6341076",
    port = "5432"
    )
    con.autocommit = True
    cursor = con.cursor()
    print("Verifique el SRID y el proj4text de la tabla PAN_pais.")
    cursor.execute("""select * from spatial_ref_sys where srid = '5472';""")
    rows = cursor.fetchall()
    result = []
    for row in rows:
      for x in row:
        result.append(str(x))

    r = rep.Repr()
    r.maxlist = 1       # max elements displayed for lists
    r.maxstring = 28    # max characters displayed for strings

    result2 =  PrettyTable()
    result2.field_names = ["srid","auth_name","auth_srid","srtext","proj4text"]
    result2.add_row([result[0],result[1],result[2],r.repr(result[3]),r.repr(result[4])])
    print(result2)


    print("¿Cuantos tipos de caminos existen en la tabla Pan_roads? ¿Cuántos hay para cada tipo? (agrupe con Groupby)")
    cursor.execute("""select count(rtt_descri) as numero_caminos, rtt_descri as tipo_camino from pan_roads group by rtt_descri;""")
    result = from_db_cursor(cursor)
    print(result)

    print("Liste la información de aquellos distritos, de la tabla Pan_adm3, que estén en comarca indígena o los que están en o cerca de un cuerpo de agua")
    cursor.execute("""select gid,iso, name_0 as pais,id_1 as cod_prov, name_1 as provincia,id_2,name_2 as distrito,name_3 as lugar, type_3 as tipo  from pan_adm3 where type_3 like 'Comarca Indígena' or type_3 like 'Water body';""")
    result = from_db_cursor(cursor)
    print(result)

    print("Calcule el área en km 2 totales, usando ST_Area, para todas las provincias, ordene sus resultados de manera descendente. Compare sus resultados con la tabla en https://en.wikipedia.org/wiki/Provinces_of_Panama")
    cursor.execute("""select sum(ST_Area(geom::geography))/1000000 Area, name_1 as provincia from pan_adm2 group by name_1 order by Area desc;""")
    result = from_db_cursor(cursor)
    print(result)


    print("Provea la suma total en km de todos los segmentos de carretera que están en la tabla Pan_roads")
    cursor.execute("""select sum(ST_Length(geom::geography))/1000 km_totales from pan_roads;""")
    result = from_db_cursor(cursor)
    print(result)


    print("Liste 5 distritos que contienen aros (rings) concéntricos dentro de su área.")
    cursor.execute("""select ST_NRings(geom) as NRings, name_2 from pan_adm2 as distrito order by NRings desc limit 5;""")
    result = from_db_cursor(cursor)
    print(result)



    con.close()
except Exception as error:
    print(error)

Verifique el SRID y el proj4text de la tabla PAN_pais.
+------+-----------+-----------+------------------------------+------------------------------+
| srid | auth_name | auth_srid |            srtext            |          proj4text           |
+------+-----------+-----------+------------------------------+------------------------------+
| 5472 |    EPSG   |    5472   | 'PROJCS["Pan...SG","5472"]]' | '+proj=poly ...62 +no_defs ' |
+------+-----------+-----------+------------------------------+------------------------------+
¿Cuantos tipos de caminos existen en la tabla Pan_roads? ¿Cuántos hay para cada tipo? (agrupe con Groupby)
+----------------+-----------------+
| numero_caminos |   tipo_camino   |
+----------------+-----------------+
|       13       |     Unknown     |
|      124       |  Primary Route  |
|      229       | Secondary Route |
+----------------+-----------------+
Liste la información de aquellos distritos, de la tabla Pan_adm3, que estén en comarca indígena o los qu